In [1]:
import os
import glob
import json

def clean_key(k):
    return k.split("_", 1)[1]

base_path = "../../../evaluations/lince/lince_sentiment/dev/"

evaluation_paths = glob.glob(f"{base_path}/*.json")

models = {}

for path in evaluation_paths:
    name = os.path.basename(path).split(".")[0]
    with open(path) as f:
        model_evaluation = json.load(f)
        clean_evaluations = []
        for task in model_evaluation["evaluations"].keys():
            task_evaluations = model_evaluation["evaluations"][task]
            clean_evaluations = [
                {clean_key(k): v for k, v in ev.items()} 
                for ev in task_evaluations
            ]

            model_evaluation["evaluations"][task] = clean_evaluations
        models[name] = model_evaluation
        


In [2]:

results = []

for model, model_evaluation in models.items():
    print("="*50)
    print(model)
    for task, task_evaluations in model_evaluation["evaluations"].items():
        print(task)
        print(len(task_evaluations))
        for evaluation in task_evaluations:

            results.append({
                **{
                    "model": model,
                    "task": task,
                },
                **evaluation,
            })

robertuito-base-deacc
lince_sentiment
2
robertuito-base-cased
lince_sentiment
2


In [7]:
import pandas as pd

pd.options.display.float_format = '{:.3f}'.format

mean_df = pd.DataFrame(results).groupby(["model"]).mean()
std_df = pd.DataFrame(results).groupby(["model"]).std()
# Magia negra

#mean_df = mean_df.unstack(1)
#std_df = std_df.unstack(1)
print(mean_df.columns)

cols = ["micro_f1", "macro_f1"]
other_f1_cols = [c for c in mean_df.columns if "f1" in c and c not in cols]

idx = mean_df.index

mean_df.loc[idx, cols].round(4).astype(str) + " ± " + std_df.loc[idx, cols].round(3).astype(str)

#mean_df.loc[idx, cols].sort_values(by=["micro_f1"], ascending=False)

Index(['loss', 'negative_f1', 'negative_precision', 'negative_recall',
       'neutral_f1', 'neutral_precision', 'neutral_recall', 'positive_f1',
       'positive_precision', 'positive_recall', 'micro_f1', 'acc', 'macro_f1',
       'macro_precision', 'macro_recall', 'runtime', 'samples_per_second',
       'steps_per_second'],
      dtype='object')


,micro_f1,macro_f1
model,,
robertuito-base-cased,0.5966 ± 0.007,0.5392 ± 0.004
robertuito-base-deacc,0.5802 ± 0.003,0.5255 ± 0.004


In [6]:
pd.options.display.max_columns = None
mean_df.loc[idx, cols + other_f1_cols].sort_values(by=["macro_f1"], ascending=False)

,micro_f1,macro_f1,negative_f1,neutral_f1,positive_f1
model,,,,,
robertuito-base-cased,0.597,0.539,0.559,0.346,0.713
robertuito-base-deacc,0.580,0.525,0.547,0.333,0.696
